In [1]:
import pandas as pd
import plotly.express as px


In [6]:

df_raw = pd.read_json(r'C:\Users\breno\Documents\Programacao\Projetos\notion-analytics\src\data\parsed\parsed_analytics_filtered_False.json')
df_raw.head(3)

,id,url,page_title,alternative_title,ds,unique_views,total_views
0,31e8adb3-ae66-4b26-9ac5-f9dd5bdd4f21,https://www.notion.so/Beto-Sicupira-Funda-o-Br...,None,Beto Sicupira (Fundação Brava),2023-10-06,1,1
1,6db2925b-a3bc-4ee9-89ca-003d21204282,https://www.notion.so/Rafael-Clemente-6db2925b...,None,Rafael Clemente,2023-10-04,1,1
2,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-02,5,6


In [7]:
df = (
    df_raw
    .assign(date_visited=pd.to_datetime(df_raw['ds']))
    .sort_values(by='date_visited')
)
df.head(3)

,id,url,page_title,alternative_title,ds,unique_views,total_views,date_visited
2,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-02,5,6,2022-08-02
1037,5eed6af8-3607-4578-bea8-7ee1203d64f4,https://www.notion.so/Assessoria-Executiva-5ee...,Assessoria Executiva,Assessoria Executiva,2022-08-02,1,2,2022-08-02
3180,39e0ac46-0eab-4b85-8ac8-f3a2509cab71,https://www.notion.so/Victor-Lameiras-39e0ac46...,None,Victor Lameiras,2022-08-02,1,2,2022-08-02


In [8]:
df.dtypes


id                           object
url                          object
page_title                   object
alternative_title            object
ds                           object
unique_views                  int64
total_views                   int64
date_visited         datetime64[ns]
dtype: object

In [9]:
# Aggregate the data by month
df['month_visited'] = df['date_visited'].dt.to_period('M').astype(str)
df_monthly = df.groupby(['month_visited', 'page_title']).agg({
    'unique_views': 'sum',
    'total_views': 'sum'
}).reset_index()

df_monthly.head()

,month_visited,page_title,unique_views,total_views
0,2022-08,Alterações do site,6,9
1,2022-08,Assessoria Executiva,2,3
2,2022-08,Benchmarks,1,1
3,2022-08,Brainstorms - Infos doadores,6,7
4,2022-08,Calendário de Comunicação,8,10


In [25]:
df_monthly.query('month_visited >= "2023-08"')

,month_visited,page_title,unique_views,total_views
724,2023-08,,2,3
725,2023-08,Acompanhamento de Projetos,6,16
726,2023-08,Anotações Reunião de Área,1,1
727,2023-08,Assessoria Executiva,6,13
728,2023-08,Atas de reunião,2,3
...,...,...,...,...
957,2023-10,Sugestões de Aprendizado,1,1
958,2023-10,Tecnologia,3,3
959,2023-10,Templates,1,1
960,2023-10,Tutoriais,4,4


In [38]:
from datetime import date
from dateutil.relativedelta import relativedelta


MONTHS_AGO = 3
str(date.today().month - MONTHS_AGO).zfill(2)

str(date.today() - relativedelta(months=3, day=1))

'2023-07-01'

In [42]:
date_to_filter.strftime('%Y-%m')

'2023-07'

In [40]:
df_monthly.query(f'month_visited >= "{date_to_filter}"')

,month_visited,page_title,unique_views,total_views
724,2023-08,,2,3
725,2023-08,Acompanhamento de Projetos,6,16
726,2023-08,Anotações Reunião de Área,1,1
727,2023-08,Assessoria Executiva,6,13
728,2023-08,Atas de reunião,2,3
...,...,...,...,...
957,2023-10,Sugestões de Aprendizado,1,1
958,2023-10,Tecnologia,3,3
959,2023-10,Templates,1,1
960,2023-10,Tutoriais,4,4


In [44]:
date_to_filter = (date.today() - relativedelta(months=3, day=1)).strftime('%Y-%m')
legend_order = df_monthly.query(f'month_visited >= "{date_to_filter}"').groupby('page_title')['unique_views'].sum().sort_values(ascending=False).index.tolist()
fig_unique_monthly = px.line(df_monthly, x='month_visited', y='unique_views', color='page_title', title='Visualizações Únicas por Mês', category_orders={"page_title": legend_order})
fig_unique_monthly

In [45]:
# Reordering the legend based on the sum of 'total_views' for each 'page_title'
legend_order = df_monthly.query(f'month_visited >= "{date_to_filter}"').groupby('page_title')['total_views'].sum().sort_values(ascending=False).index.tolist()

# Creating the line plot with custom legend order based on 'total_views'
fig_total_monthly = px.line(df_monthly, x='month_visited', y='total_views', color='page_title', 
                            title='Visualizações Totais por Mês',
                            category_orders={"page_title": legend_order})

# Show the figure
fig_total_monthly.show()


In [46]:
df_total = df_monthly.groupby(['month_visited'])['unique_views'].sum().reset_index()

fig_total_sum_monthly = px.line(df_total, x='month_visited', y='unique_views', 
                            title='Visualizações Únicas por Mês Global')
fig_total_sum_monthly.show()